In [1]:
import gym
from gym_power_trading.envs import PowerTradingEnv
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

In [2]:
file_name = 'data\AEP_PSGC1_AMP_long.h5'
path = Path(file_name).resolve()
uri = str(path)

In [3]:
df = pd.read_hdf(uri)
df.head()

HE
MARKET_DAY NODE          TYPE    VALUE HourEnding       
2021-07-01 AEP.PSGC1.AMP Gennode LMP   1           25.88
                                       2           24.88
                                       3           23.45
                                       4           22.47
                                       5           21.73

In [2]:
my_dataframe = pd.read_csv("RandomCloseValues.csv")

In [3]:
env = PowerTradingEnv(df=my_dataframe, window_size=3, frame_bound=(10, 100))
obs = env.reset()
for _ in tqdm(range(100)):
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)
    print(info)
    if truncated:
        obs = env.reset()

100%|██████████| 100/100 [00:00<00:00, 19066.75it/s]

{'total_reward': 5.109017944335932, 'total_profit': 0.0, 'position': 1, 'battery_charge': 20}
{'total_reward': -372.0057281494139, 'total_profit': -375.2291723632811, 'position': 0, 'battery_charge': 0}
{'total_reward': -370.88963966369613, 'total_profit': -375.2291723632811, 'position': 1, 'battery_charge': 20}
{'total_reward': -454.2396991729735, 'total_profit': -458.1624815750121, 'position': 0, 'battery_charge': 0}
{'total_reward': -454.2396991729735, 'total_profit': -458.1624815750121, 'position': 2, 'battery_charge': 0}
{'total_reward': -454.2396991729735, 'total_profit': -458.1624815750121, 'position': 2, 'battery_charge': 0}
{'total_reward': -454.2396991729735, 'total_profit': -458.1624815750121, 'position': 2, 'battery_charge': 0}
{'total_reward': -454.2396991729735, 'total_profit': -458.1624815750121, 'position': 0, 'battery_charge': 0.0}
{'total_reward': -454.2396991729735, 'total_profit': -458.1624815750121, 'position': 0, 'battery_charge': 0.0}
{'total_reward': -454.239699

In [4]:
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_checker import check_env

DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7



In [5]:
check_env(env)

In [6]:
venv = make_vec_env(lambda: PowerTradingEnv(df=my_dataframe, window_size=3, frame_bound=(10, 100)))

### Need to Modify pre-processing function, otherwise self._observation_space produces mismatch produces broadcast error during learning

In [7]:
model = DQN('MlpPolicy', venv)
model.learn(total_timesteps=10000,
            log_interval=10000,
            progress_bar=True)

Output()

DEBUG:Comm:handle_msg[c02b12d4752d4325acfb22154fb108b7]({'header': {'date': datetime.datetime(2024, 4, 18, 2, 58, 59, 756000, tzinfo=tzutc()), 'msg_id': 'ab143f88-7010-422b-90a3-431433ce38a7', 'msg_type': 'comm_msg', 'session': '8079b571-7e49-4f21-9294-c5284bf6eb23', 'username': '264293a7-f70f-496f-9a7e-4703cf023644', 'version': '5.2'}, 'msg_id': 'ab143f88-7010-422b-90a3-431433ce38a7', 'msg_type': 'comm_msg', 'parent_header': {}, 'metadata': {}, 'content': {'comm_id': 'c02b12d4752d4325acfb22154fb108b7', 'data': {'method': 'update', 'state': {'outputs': [{'output_type': 'display_data', 'data': {'text/plain': '\x1b  86%\x1b \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b\x1b━━━━━━━━\x1b \x1b8,569/10,000 \x1b [ \x1b0:00:00\x1b < \x1b0:00:01\x1b , \x1b10,616 it/s\x1b ]\n', 'text/html': '<pre style="white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,\'DejaVu Sans Mono\',consolas,\'Courier New\',monospace"><span style="color: #800080; text-decoration-col

In [13]:
from collections import deque

x = deque(maxlen=5)
x.append([1, 2, 3])

In [18]:
np.flip(np.array(x).reshape(-1, 1))

array([[3],
       [2],
       [1]])